# IGN HD liDAR PCD Generation

In [ ]:
import sys
import os
sys.path.append('../src/util/')
from general import IGNLidarProcessor, PointCloudHandler


input_file = "../src/dem-data/LHD_FXX_0982_6524_PTS_C_LAMB93_IGN69.copc.laz"
sample_fraction = 0.2

processor = IGNLidarProcessor(input_file)
processor.read_point_cloud()
processor.transform_coordinates()
processor.generate_color()

In [5]:
processor.df

,x,y,z,classification,color
0,315679.346779,5.069587e+06,1617.46,2,"(31, 119, 180)"
1,315686.236590,5.069588e+06,1612.30,2,"(31, 119, 180)"
2,315684.084498,5.069587e+06,1613.96,3,"(255, 127, 14)"
3,315682.723714,5.069587e+06,1617.51,5,"(214, 39, 40)"
4,315679.708940,5.069587e+06,1617.06,3,"(255, 127, 14)"
...,...,...,...,...,...
33210848,315583.085407,5.068814e+06,1743.79,3,"(255, 127, 14)"
33210849,315580.806897,5.068815e+06,1743.52,2,"(31, 119, 180)"
33210850,315547.930500,5.068845e+06,1735.67,3,"(255, 127, 14)"
33210851,315539.459305,5.068842e+06,1740.04,2,"(31, 119, 180)"


In [ ]:
handler = PointCloudHandler(processor.df)
handler.downsample(sample_fraction=sample_fraction)
handler.to_open3d()
handler.save_point_cloud(filename="IGN_LiDAR_HD.ply")

In [ ]:
import sys
import os
sys.path.append('../src/util/')
from general import Sentinel2Reader, load_dem_utm, PcdGenerator, PointCloudHandler

In [ ]:
token = os.environ.get('hdb_token')
product_path = "/home/ubuntu/project/destine-godot-mvp/src/sentinel2-data/T32TLR_20241030T103151_TCI_20m.jp2"
reader = Sentinel2Reader(filepath=product_path, preprocess=True)
bounds = reader.bounds
width = reader.width
height = reader.height
parameter = 'dem'
sample_fraction = 100


dem_data = load_dem_utm(token, parameter, bounds, width, height)
# Initialize and generate point cloud
pcd_gen = PcdGenerator(sat_data=reader.data,
                       dem_data=dem_data,
                       sample_fraction=sample_fraction)
pcd_gen.generate_point_cloud()
pcd_gen.downsample(sample_fraction=sample_fraction)

In [6]:
pcd_gen.df

,x,y,z,color
16748987,389750,5051210,2036.6875,"(76, 42, 20)"
13426812,375250,5039110,2657.1250,"(139, 120, 88)"
28212858,404770,5092970,2865.7500,"(117, 93, 68)"
17065522,352050,5052370,3052.6250,"(113, 159, 215)"
22002682,385050,5070350,1472.9375,"(39, 32, 21)"
...,...,...,...,...
15114771,316030,5045270,1300.8750,"(24, 22, 0)"
25239592,341250,5082150,2935.3750,"(234, 255, 255)"
18505978,393570,5057610,1715.3125,"(53, 37, 14)"
26858681,332030,5088050,2228.5000,"(38, 38, 21)"


In [ ]:
import pandas as pd

data = pd.concat([pcd_gen.df, processor.df], axis=0)
handler = PointCloudHandler(data)
handler.to_open3d()
handler.save_point_cloud(filename="New_IGN_LiDAR_HD.ply")